In [78]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

In [79]:
data = pd.read_csv(r'C:\Users\varun\OneDrive\Desktop\takehome_user_engagement_1.csv')

In [80]:
data_1 = pd.read_excel(r'C:\Users\varun\OneDrive\Desktop\takehome_users_2.xlsx')

In [81]:
data_1

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:00,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:00,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:00,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:00,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:00,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
...,...,...,...,...,...,...,...,...,...,...
11995,11996,2013-09-06 06:14:00,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0
11996,11997,2013-01-10 18:28:00,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11997,11998,2014-04-27 12:45:00,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0
11998,11999,2012-05-31 11:55:00,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,1.338638e+09,0,0,6,NaN


In [82]:



# Create DataFrame
df = pd.DataFrame(data)

# Convert 'time_stamp' to datetime
df['time_stamp'] = pd.to_datetime(df['time_stamp'], format='%d-%m-%Y %H:%M')

# Sort by 'user_id' and 'time_stamp'
df = df.sort_values(by=['user_id', 'time_stamp'])

# Function to check visits on different days in the last 7 days
def check_visits_in_last_7_days(df):
    df['visited_in_7_days'] = 0
    for user_id in df['user_id'].unique():
        user_visits = df[df['user_id'] == user_id]
        set_visited_in_7_days = False
        for i in range(len(user_visits)):
            start_date = user_visits.iloc[i]['time_stamp']
            end_date = start_date - pd.Timedelta(days=7)
            visits_in_window = user_visits[(user_visits['time_stamp'] <= start_date) & 
                                           (user_visits['time_stamp'] > end_date)]
            unique_days = visits_in_window['time_stamp'].dt.date.nunique()
            if unique_days >= 3:
                set_visited_in_7_days = True
                break
        if set_visited_in_7_days:
            df.loc[user_visits.index, 'visited_in_7_days'] = 1
    return df

# Apply the function
df = check_visits_in_last_7_days(df)

# Save the DataFrame to a CSV file
df.to_csv('result.csv', index=False)

# Print the DataFrame
print(df)


                time_stamp  user_id  visited  visited_in_7_days
0      2014-04-22 03:53:00        1        1                  0
1      2013-11-15 03:45:00        2        1                  1
2      2013-11-29 03:45:00        2        1                  1
3      2013-12-09 03:45:00        2        1                  1
4      2013-12-25 03:45:00        2        1                  1
...                    ...      ...      ...                ...
207912 2013-09-06 06:14:00    11996        1                  0
207913 2013-01-15 18:28:00    11997        1                  0
207914 2014-04-27 12:45:00    11998        1                  0
207915 2012-06-02 11:55:00    11999        1                  0
207916 2014-01-26 08:57:00    12000        1                  0

[207917 rows x 4 columns]


In [83]:
df.to_csv('result_1.csv', index=False)
df = df[['user_id', 'visited_in_7_days']]


In [84]:
merged_data = pd.merge(df, data_1, left_on='user_id', right_on='invited_by_user_id', how='inner')


In [85]:
merged_data['visited_in_7_days']

0         0
1         0
2         0
3         0
4         0
         ..
280616    0
280617    0
280618    0
280619    0
280620    0
Name: visited_in_7_days, Length: 280621, dtype: int64

In [86]:
visited_counts = merged_data['visited_in_7_days'].value_counts()
visited_counts

visited_in_7_days
1    275569
0      5052
Name: count, dtype: int64

In [70]:
# Upsample minority class
minority_class_upsampled = resample(minority_class,
                                    replace=True,  # sample with replacement
                                    n_samples=len(majority_class),  # match majority class count
                                    random_state=42)  # reproducible results

# Combine majority class with upsampled minority class
balanced_data = pd.concat([majority_class, minority_class_upsampled])

# Display new class counts
print(balanced_data['visited_in_7_days'].value_counts())

visited_in_7_days
1    275569
0    275569
Name: count, dtype: int64


In [87]:
majority_class = merged_data[merged_data['visited_in_7_days'] == 1]
minority_class = merged_data[merged_data['visited_in_7_days'] == 0]

# Downsample majority class
majority_class_downsampled = resample(majority_class,
                                      replace=False,  # sample without replacement
                                      n_samples=len(minority_class),  # match minority class count
                                      random_state=42)  # reproducible results

# Combine minority class with downsampled majority class
balanced_data = pd.concat([minority_class, majority_class_downsampled])


In [88]:
balanced_data = balanced_data.drop(columns=['creation_time', 'email', 'name'])


In [91]:
balanced_data
print(balanced_data['visited_in_7_days'].value_counts())

visited_in_7_days
0    5052
1    5052
Name: count, dtype: int64


In [90]:
balanced_data = pd.get_dummies(balanced_data, columns=['creation_source'], drop_first=True)

# List of columns to drop (if any other columns need to be dropped, specify here)
# columns_to_drop = ['column1', 'column2']
# balanced_data = balanced_data.drop(columns=columns_to_drop)

# Separate input features (X) and target variable (y)
X = balanced_data.drop('visited_in_7_days', axis=1)
y = balanced_data['visited_in_7_days']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build and train the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_scaled, y_train)

# Make predictions
y_pred = clf.predict(X_test_scaled)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Check feature importance
feature_importances = clf.feature_importances_
features = X.columns

# Create a DataFrame for feature importances
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importance_df)

Accuracy: 0.9722909450766947
Confusion Matrix:
 [[974  37]
 [ 19 991]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97      1011
           1       0.96      0.98      0.97      1010

    accuracy                           0.97      2021
   macro avg       0.97      0.97      0.97      2021
weighted avg       0.97      0.97      0.97      2021


Feature Importances:
                      Feature  Importance
5                      org_id    0.248024
0                     user_id    0.203982
6          invited_by_user_id    0.200872
1                   object_id    0.159165
2  last_session_creation_time    0.147089
7  creation_source_ORG_INVITE    0.015504
3    opted_in_to_mailing_list    0.013969
4  enabled_for_marketing_drip    0.011395
